## Compile outputs

This script concatenates pyGluCEST output tsvs for the 7 network parcellation

### Import Packages

In [11]:
import os
import glob
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import pearsonr
from scipy.stats import linregress
import seaborn as sns
import matplotlib.pyplot as plt
import re
import shutil

# Set paths and variables
cestpath = "/Users/pecsok/Desktop/ImageData/PMACS_remote/data/cest/"
batches = ['output_measures/UNI', 'analysis_batch2/output_measures/UNI']
networks = ["Cont", "Default", "DorsAttn", "Vis", "SalVentAttn", "SomMot", "Limbic"] 

In [12]:
# Loop through networks and import original CEST output file
for network in networks:
    for batch in batches:
        cest_pattern = cestpath + batch + "/" + "all-GluCEST-s100_7-" + network + "-measures_UNI.tsv"
        cestfile = glob.glob(cest_pattern)
        for file in cestfile:
            if os.path.isfile(file):
                cestmat = pd.read_csv(file, sep='\t')
                
